In [55]:
import numpy as np
import queue
from functools import reduce

In [56]:
def make_chains(mat, names, k, key):
    
    def dfs(que, mat, v, k_now, k_max, chainsDict, names):
        print(len(chainsDict))
        if k_now == k_max:
            name_in_num = list(que)
            name_in_num.sort()
            name_in_num = list(map(lambda i: names[i], name_in_num))
            kk = reduce(lambda i, j: i + j, name_in_num)
            chainsDict.setdefault(kk, 0)
            chainsDict[kk] += 1
            return
        
        
        for i in range(0, mat.shape[0]):
            if mat[v, i] == 1 and not i in que:
                que.append(i)
                dfs(que, mat, i, k_now + 1, k_max, chainsDict, names)
                del(que[k_now])
                
            
    chainsDict = dict.fromkeys(key, 0)
    
    for i in range(mat.shape[0]):#Запускаем обход в глубину k из каждой вершины
        que = []
        que.append(i)
        
        dfs(que, mat, i, 1, k, chainsDict, names)
        del(que[0])
        
        
    for i in chainsDict.keys(): #Каждую цепочку посчитали два раза
        chainsDict[i] //= 2
    return chainsDict
        

In [57]:
def getAllKChainsNameInFile(filename, k): #k-размер цепочек
    all_chains_names = []
    ctr = 0
    with  open(filename, "r") as f:
        
        
        for i in f:
            ctr += 1
            mol_atoms = []
            mol_name = i.split()[0]
            num_elems = int(i.split()[1])
            

            matrix = np.zeros((num_elems, num_elems))
            for i in range(num_elems):
                line = next(f) #Берём i линию в молекуле
                mol_atoms.append(line.split()[0]) #По первому индексу хранятся названия, которые пихаем в лист

                it = 3 #С третьего элемента в строке начинаются связи
                while line.split()[it] != "000":
                    matrix[i, int(line.split()[it][:-1]) - 1] = 1 #int(line.split()[it][-1]) Если захотим тип связи
                    matrix[int(line.split()[it][:-1]) - 1, i] = 1
                    it += 1
            res = make_chains(matrix, mol_atoms, k, {})
            all_chains_names += res.keys()
          
    return all_chains_names, ctr

In [58]:
filename = "GLASS.STR"

In [59]:
f = open(filename, "r")

In [60]:
names = getAllKChainsNameInFile(filename, 3)

0
0
0
1
1
1
1
2
2
2
2
2
3
4
4
4
4
4
4
4
4
4
5
5
5
5
0
0
0
1
1
1
1
2
2
2
2
2
3
4
4
4
4
4
5
5
5
5
5
6
6
6
6
6
6
6
6
0
0
0
1
2
2
2
2
3
4
4
4
4
4
5
5
5
5
5
5
6
6
6
6
6
6
6
6
0
0
0
1
1
2
2
2
3
3
4
4
4
4
4
5
6
6
6
6
6
7
7
8
8
8
8
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
0
0
0
1
1
2
2
2
3
3
4
4
4
4
4
5
6
6
6
6
6
7
7
8
9
9
9
9
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
11
11
11
11
0
0
0
1
1
2
2
2
3
3
4
4
4
4
4
5
6
6
6
6
6
7
7
8
8
8
8
9
9
9
9
9
9
9
9
9
9
9
9
9
10
10
10
10
10
11
11
11
11
11
12
12
12
12
12
13
13
13
13
13
13
13
0
0
0
1
1
2
2
2
3
3
4
4
4
4
4
5
6
6
6
6
6
7
7
7
7
7
8
8
8
8
8
8
8
8
8
8
8
0
0
0
1
1
2
2
2
3
3
4
4
4
4
4
5
6
6
6
6
6
7
7
8
9
9
9
9
10
10
10
10
10
10
10
10
10
10
10
10
10
10
11
11
11
11
12
12
12
12
12
12
12
12
0
0
0
1
1
2
2
2
3
3
4
4
4
4
4
5
6
6
6
6
6
7
7
8
8
8
8
9
9
9
9
9
9
9
9
9
9
9
9
9
10
10
10
10
10
11
11
11
11
11
11
11
0
0
0
1
1
2
2
2
3
3
4
4
4
4
4
5
6
6
6
6
6
7
7
8
8
8
8
9
9
9
9
9
9
9
9
9
9
9
9
9
10
10
10
10
10
11
11
11
11
11
12
12
12
12
12
12
12
0
0
0
1
1
1
1
2
2
2
2
2
3


59
60
60
60
60
61
61
62
63
63
63
63
63
63
63
64
65
65
65
65
65
65
65
66
66
66
66
66
66
67
67
68
68
68
68
69
69
70
71
71
71
71
71
72
72
73
74
74
74
74
75
75
76
77
77
77
77
77
77
77
78
79
79
79
79
79
79
79
80
80
80
80
80
80
81
81
82
82
82
82
83
83
84
84
84
84
84
85
85
85
85
85
86
86
86
86
86
86
86
86
86
86
86
86
86
86
86
86
86
87
87
88
88
88
88
89
89
90
90
90
90
90
91
91
91
91
91
92
92
92
92
92
92
92
92
92
92
92
92
92
92
92
92
92
93
93
94
94
94
94
95
95
96
96
96
96
96
97
97
97
97
97
98
98
98
98
98
98
98
98
98
98
98
98
98
98
98
98
98
99
99
100
100
100
100
101
101
102
102
102
102
102
103
103
103
103
103
104
104
104
104
104
104
104
104
104
104
104
0
0
0
1
2
3
3
3
3
3
4
5
5
6
7
7
7
7
8
9
9
9
9
9
10
10
11
11
12
12
12
12
13
13
14
15
15
15
15
15
16
17
17
17
17
17
18
18
19
20
20
21
21
21
21
22
22
22
22
22
22
22
22
22
22
23
23
23
23
23
23
23
24
24
25
25
25
25
26
26
27
28
28
28
28
28
29
30
30
30
30
30
31
31
32
33
33
34
34
34
34
35
35
35
35
35
35
35
35
35
35
36
36
36
36
36
36
37
38
38
38
38
38
38
3

38
38
38
39
39
40
40
40
40
40
41
42
42
42
42
42
43
43
44
45
45
45
45
46
46
46
46
46
46
46
46
46
46
46
46
46
46
47
48
48
49
50
50
50
50
50
50
51
52
52
53
54
54
54
54
55
55
56
56
57
57
57
57
58
58
59
59
59
59
59
60
60
60
60
60
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
62
62
63
63
63
63
64
64
65
65
65
65
65
66
66
66
66
66
67
67
67
67
67
67
67
67
67
67
67
67
67
67
67
68
68
69
69
70
70
70
70
71
71
72
72
72
72
72
73
73
73
73
73
74
74
74
74
74
74
74
74
74
74
74
74
74
74
74
74
74
75
75
76
76
76
76
77
77
78
78
78
78
78
79
79
79
79
79
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
81
81
82
82
82
82
82
82
83
83
83
84
84
84
84
84
84
84
84
84
85
85
85
85
85
85
86
86
86
86
86
86
86
86
86
86
0
0
0
1
2
2
3
4
4
5
6
6
6
7
8
8
9
9
10
10
10
10
11
11
12
12
12
12
12
13
14
14
14
14
14
15
15
16
17
17
17
17
18
18
18
18
18
18
18
18
18
18
18
18
19
19
20
20
21
21
21
21
22
22
23
23
23
23
23
24
25
25
25
25
25
26
26
27
28
28
28
28
29
29
29
29
29
29
29
29
29
29
29
29
29
29
30
30
31
31
31
31
32
32
33
33
33
33
3

0
0
0
1
1
2
2
3
4
4
4
5
6
6
7
8
8
8
8
8
9
9
10
11
11
11
11
12
12
12
12
12
12
13
13
13
13
13
14
14
15
15
16
17
17
17
18
19
19
20
20
20
20
20
21
22
22
22
22
22
22
22
23
24
24
24
24
25
25
25
26
26
26
26
26
26
27
27
28
28
28
28
29
29
30
30
30
30
30
31
32
32
32
32
32
33
33
34
35
35
35
35
36
36
36
36
36
36
36
36
36
36
36
36
36
36
37
37
38
38
38
38
39
39
40
40
40
40
40
41
42
42
42
42
42
43
43
44
45
45
45
45
46
46
46
46
46
46
46
46
46
46
46
46
46
46
46
46
46
46
46
46
46
46
46
47
48
48
49
50
50
51
51
52
52
52
53
54
55
55
56
57
57
58
58
58
58
59
59
60
61
61
62
62
62
62
63
63
63
64
64
65
65
65
65
66
67
67
67
68
68
69
69
69
69
69
69
69
70
70
70
70
70
70
70
70
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
72
72
72
72
72
72
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
74
74
75
75
75
75
75
76
76
76
76
76
77
77
77
77
77
78
78
78
78
78
79
79
79
79
79
80
80
80
80
80
80
80
81
81
81
81
81
82
82
82
82
82
83
83
83
83
83
84
84
84
84
84
85
85
85
85
85
86
86
86
86
87
87
87
87
87
87
87
87

In [52]:
def getMatrix(filename, k, keys, high):
    res_matr = np.zeros((high, len(keys)))
    dic = dict.fromkeys(keys, 0)
    cnt = 0
    with open(filename, "r") as f:
        for i in f:
            
            mol_atoms = []
            num_elems = int(i.split()[1])
            matrix = np.zeros((num_elems, num_elems))
            for i in range(num_elems):
                line = next(f) #Берём i линию в молекуле
                mol_atoms.append(line.split()[0]) #По первому индексу хранятся названия, которые пихаем в лист

                it = 3 #С третьего элемента в строке начинаются связи
                while line.split()[it] != "000":
                    matrix[i, int(line.split()[it][:-1]) - 1] = 1 #int(line.split()[it][-1]) Если захотим тип связи
                    matrix[int(line.split()[it][:-1]) - 1, i] = 1
                    it += 1
            res = make_chains(matrix, mol_atoms, k, dic)
            print(len(keys))
            res_matr[i,:] = list(dic.values())[:]
            cnt += 1
    return res_matr
        
    

In [53]:
Matr = getMatrix(filename, 3, names[0], names[1])

993
993
993
993
993
993
993
993
993
993
993
993
993
993
993
993
993
993
993
993
993
993
993
993
993
993
3113


ValueError: cannot copy sequence with size 993 to array axis with dimension 3113

In [74]:
names[1]

103